# Capítulo 6 - SARSA de _n_ passos e Ambientes Contínuos


Você pode rodar este notebook localmente ou no Colab. Para abrir diretamente no Colab, basta clicar no link abaixo.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap06novo/cap06-main.ipynb)

## Configurações Iniciais

In [ ]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # for saving videos
    !apt-get install ffmpeg

    !pip install gymnasium
    !pip install optuna

    # clone repository
    !git clone https://github.com/pablo-sampaio/rl_facil
    sys.path.append("/content/rl_facil")

    clear_output()
else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

clear_output()

In [ ]:
'''if IN_COLAB:
    # Set up fake display; otherwise rendering will fail
    import os
    os.system("Xvfb :1 -screen 0 1024x768x24 &")
    os.environ['DISPLAY'] = ':1'


'''

In [ ]:
import gymnasium as gym
import numpy as np
import optuna

from util.experiments import repeated_exec
from util.plot import plot_result, plot_multiple_results
from util.wrappers import DiscreteObservationWrapper
from util.notebook import display_videos_from_path
from util.qtable_helper import evaluate_qtable

from cap04.montecarlo_v2 import run_montecarlo2
from cap05.qlearning_sarsa import run_qlearning
from cap05.expected_sarsa import run_expected_sarsa

from nstep_sarsa import run_nstep_sarsa

## 1 - SARSA de n Passos

É uma extensão do SARSA que usa uma sequência de de *n* passos (ou seja, com *n* ações realizadas no ambiente) como entrada para cada atualização da estimativa do $Q(s,a)$.

Com **n=1**, ele estima $Q(s,a)$ igual ao *SARSA*:
- usa esta experiência: $s, a, r_1, s_1, a_1$
- calcula a nova estimativa de $Q(s,a)$ assim (para estados não-terminais):
$$Q_{target} = r_1 + \gamma . Q(s_1,a_1)$$
- note que apenas 1 ação foi realizada ($a$) e apenas 1 recompensa foi recebida ($r_1$)
- observe que a última ação ($a_1$) foi escolhida, mas não foi realizada no ambiente

Com **n=2**, para estimar $Q(s,a)$, ele:
- usa esta experiência: $s, a, r_1, s_1, a_1, r_2, s_2, a_2$
- calcula a nova estimativa de $Q(s,a)$ assim:
$$Q_{target} = r_1 + \gamma .r_2 + \gamma^2 . Q(s_2,a_2)$$
- esta é uma estimativa mais precisa, que usa os dados de 2 recompensas reais

Para **n qualquer**:
- experiência: $s, a, r_1, s_1, a_1, r_2, s_2, a_2, s_3, \cdots, r_n, s_n, a_n$
- nova estimativa:
$$Q_{target} = r_1 + \gamma .r_2 + \gamma^2 . r_2 + \cdots + \gamma^n . Q(s_n,a_n)$$

O **n** será um parâmetro do algoritmo.

O código é mais complexo do que dos algoritmos de 1 passo. Veja o arquivo `cap05/run_nstep_sarsa.py` para conhecer os detalhes.

Abaixo, vamos importar e usar aquela implementação para fazer um experimento:

In [ ]:
#ENV_NAME, r_max = "Taxi-v3", 10
ENV_NAME, r_max = "CliffWalking-v0", 0

EPISODES = 7000
LR = 0.01
GAMMA = 0.95
EPSILON = 0.1
NSTEPS = 3

env = gym.make(ENV_NAME)

# Roda o algoritmo "n-step SARSA"
rewards, qtable = run_nstep_sarsa(env, EPISODES, NSTEPS, LR, GAMMA, EPSILON, render=False)
print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

In [ ]:
# Mostra um gráfico de episódios x retornos não descontados
plot_result(rewards, r_max, None)

In [ ]:
evaluate_qtable(env, qtable, 1, verbose=True);

In [ ]:
RUNS = 5
results = []
for nstep in [1, 2, 3]:
    results.append( repeated_exec(RUNS, f"{nstep}-step SARSA (LR={LR})", run_nstep_sarsa, env, EPISODES, nstep, LR) )
    clear_output()

In [ ]:
plot_multiple_results(results, cumulative=False, x_log_scale=True)

## 2 - Lidando com Ambientes Contínuos

Vamos usar os mesmos algoritmos de antes, baseados em Q-Table, para lidar com ambientes de estados contínuos. 

Para isso, vamos usar um wrapper que discretiza os estados desses ambientes.

Abaixo, nós criamos um ambiente e mostramos o "formato" do seu estado e um exemplo de estado do ambiente.

In [ ]:
# 1. Criar o ambiente contínuo
ENV_NAME = "CartPole-v1"
r_max_plot = 200

env = gym.make(ENV_NAME)

# vamos ver como é um estado deste ambiente?
print("Espaço de estados/observações: ", env.observation_space)
print("  - formato: ", env.observation_space.shape)
print("  - exemplo: ", env.reset())

In [ ]:
# 2. Encapsular o ambiente em nosso wrapper

# atenção para o 2o parâmetro: deve ter um valor para cada componente do estado
env = DiscreteObservationWrapper(env, [20,50,10,20])

In [ ]:
# 3. Rodar um algoritmo de treinamento

EPISODES = 1000
LR = 0.5
GAMMA = 0.95
EPSILON = 0.1
NSTEPS = 2

#rewards, qtable = run_expected_sarsa(env, EPISODES, LR, GAMMA, EPSILON, render=False)
rewards, qtable = run_nstep_sarsa(env, EPISODES, NSTEPS, LR, GAMMA, EPSILON, render=False)

print("Últimos resultados: media =", np.mean(rewards[-20:]), ", desvio padrao =", np.std(rewards[-20:]))

In [ ]:
# 4. Gerar um gráfico de episódios x retornos (não descontados)

filename = f"results/expected_sarsa-{ENV_NAME.lower()[0:8]}-ep{EPISODES}-lr{LR}.png"
plot_result(rewards, r_max_plot, None)

In [ ]:
# 5. Faz alguns testes, usando a tabela de forma greedy

test_greedy_Q_policy(env, qtable, 10, render=False, recorded_video_folder='./videos')
env.close()

In [ ]:
display_videos_from_path('./videos')

## 3 - Otimizando Parâmetros

Vamos usar a biblioteca *Optuna* para otimizar (hiper-)parâmetros dos algoritmos de treinamento.

### 3.1 - Ambiente Discreto

Este é o caso mais simples, porque não precisamos aplicar nenhum wrapper.

Primeiro, você precisa fazer uma função que receber um parâmetro "Trial" (objeto do optuna) e retorna uma medida de desempenho.

Dentro da função, você usa o trial para pedir "sugestões" de valores para os hiper-parâmetros do seu algoritmo.

In [ ]:
ENV = gym.make("Taxi-v3")

def train(trial : optuna.Trial):
    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    lr = trial.suggest_uniform('learning_rate', 0.001, 1.0)
    #lr = trial.suggest_loguniform('learning_rate', 0.001, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)

    print(f"\nTRIAL #{trial.number}: lr={lr}, eps={eps}")

    # roda o algoritmo e recebe os retornos não-descontados
    (returns, _) = run_montecarlo2(ENV, 4000, lr=lr, epsilon=eps)

    # limpa a saída da célula do notebook
    clear_output()

    # média dos retornos dos últimos 100 episódios
    return sum(returns[-1000:])/1000


Depois, você cria um "study" do Optuna e manda otimizar sua função, indicando quantas tentativas (trials) ele vai fazer -- ou seja, quantas vezes a sua função vai ser executada.

In [ ]:
study = optuna.create_study(direction='maximize',
                        storage='sqlite:///resultado_optuna.db',
                        study_name='montecarlo2',
                        load_if_exists=True)

# maximiza o valor de retorno de train_exp_sarsa, rodando "n_trials" vezes
study.optimize(train, n_trials=10)

print("MELHORES PARÂMETROS:")
print(study.best_params)

### 3.2 - Ambiente Contínuo

Neste cao, nós aplicamos o wrapper, que tem novos parâmetros (quantidade de "bins" por valor do estado).

O wrapper é considerado parte da solução e, por isso, o ideal é tentar otimizar esses parâmetros. Por isso, o wrapper será criado dentro da função a ser otimizada.

In [ ]:
# cria apenas a versão contínua
ENV = gym.make("CartPole-v1")

def train_continuous(trial : optuna.Trial):

    # chama os métodos do "trial" (tentativa) para sugerir valores para os parâmetros
    lr = trial.suggest_uniform('learning_rate', 0.001, 1.0)
    eps = trial.suggest_uniform('epsilon', 0.01, 0.2)
    bins1 = trial.suggest_int('bins1', 5, 100)
    bins2 = trial.suggest_int('bins2', 5, 100)
    bins3 = trial.suggest_int('bins3', 5, 100)
    bins4 = trial.suggest_int('bins4', 5, 100)

    all_bins = [bins1, bins2, bins3, bins4]

    print(f"\nTRIAL #{trial.number}: lr={lr}, eps={eps}, bins={all_bins}")

    # cria o wrapper e roda o algoritmo, recebendo os retornos não-descontados
    env_wrapper = DiscreteObservationWrapper(ENV, all_bins)
    (returns, _) = run_expected_sarsa(env_wrapper, 4000, lr=lr, epsilon=eps)

    # limpa a saída da célula do notebook
    clear_output()

    # média dos retornos dos últimos 1000 episódios
    return sum(returns[-1000:])/1000


In [ ]:
study = optuna.create_study(direction='maximize',
                        storage='sqlite:///resultado_optuna.db',
                        study_name='expsarsa_continuo',
                        load_if_exists=True)

# maximiza o valor de retorno de train_exp_sarsa_continuous, rodando "n_trials" vezes
study.optimize(train_continuous, n_trials=10)

print("MELHORES PARÂMETROS:")
print(study.best_params)

## 4 - Experimentos Completos

Agora que você descobriu bons parâmetros, que tal rodar um treinamento mais longo com o seu algoritmo?

Seguem experimentos com os resultados de otimizações obtidas com os alunos do semestre 2021.2.

### Taxi-v3 (discreto)

In [ ]:
environment = gym.make("Taxi-v3")
NUM_EPISODES = 20000

results = []

# 'epsilon': 0.012630525410678125, 'learning_rate': 0.9189058377450972
results.append( repeated_exec(3, f"Q-Learning (Maely)", run_qlearning, environment, NUM_EPISODES, 0.918, 0.95, 0.012) )
clear_output()

# 'epsilon': 0.01049172026352314, 'learning_rate': 0.07158117429097055
results.append( repeated_exec(3, f"Exp-Sarsa (Bruno)", run_expected_sarsa, environment, NUM_EPISODES, 0.0715, 0.95, 0.010) )
clear_output()

# {'epsilon': 0.07291046034184917, 'learning_rate': 0.025046716210814366}
results.append( repeated_exec(3, f"MonteCarlo2 (M.Wei)", run_montecarlo2, environment, NUM_EPISODES, 0.0250, 0.95, 0.072) )
clear_output()

plot_multiple_results(results, cumulative=False, x_log_scale=False)

### FrozenLake-v1 (discreto)

In [ ]:
environment = gym.make("FrozenLake-v1")
NUM_EPISODES = 20000

results = []

# {'epsilon': 0.03081133375694635, 'learning_rate': 0.5177593676824266}
results.append( repeated_exec(3, f"Q-Learning (J.Rodrigues)", run_qlearning, environment, NUM_EPISODES, 0.517, 0.95, 0.030) )
clear_output()

# 'epsilon': 0.045332160883937794, 'learning_rate': 0.24962062732262033
results.append( repeated_exec(3, f"Exp-Sarsa (Matheus)", run_expected_sarsa, environment, NUM_EPISODES, 0.249, 0.95, 0.045) )
clear_output()

# learning_rate': 0.8054242666287916, 'epsilon': 0.07261114965386482, 'gamma': 2.2849204996994357
results.append( repeated_exec(3, f"MonteCarlo2 (L.Fernando)", run_montecarlo2, environment, NUM_EPISODES, 0.805, 2.28, 0.072) )
clear_output()

plot_multiple_results(results, cumulative=False, x_log_scale=False)

### MountainCar (contínuo)

In [ ]:
ENV = gym.make("MountainCar-v0")
NUM_EPISODES = 20000

results = []

'''
Mateus {'bins1': 56, 'bins2': 51, 'epsilon': 0.01186126148733685, 'learning_rate': 0.6912469478373227}
Daniel {'bins1': 54, 'bins2': 70, 'epsilon': 0.13703891101977922, 'learning_rate': 0.6792059384074097}
Diego {bins1': 65, 'bins2': 46, 'epsilon': 0.09108892703810256, 'learning_rate': 0.27627593841593145}
'''

wrapped_env = DiscreteObservationWrapper(ENV, [56,51])
results.append( repeated_exec(3, f"Q-Learning (Mateus)", run_qlearning, wrapped_env, NUM_EPISODES, 0.691, 0.95, 0.011) )
clear_output()

wrapped_env = DiscreteObservationWrapper(ENV, [54,70])
results.append( repeated_exec(3, f"Exp-Sarsa (Daniel)", run_expected_sarsa, wrapped_env, NUM_EPISODES, 0.679, 0.95, 0.137) )
clear_output()

wrapped_env = DiscreteObservationWrapper(ENV, [65,46])
results.append( repeated_exec(3, f"MonteCarlo2 (Diego)", run_montecarlo2, wrapped_env, NUM_EPISODES, 0.276, 2.28, 0.091) )
clear_output()

plot_multiple_results(results, cumulative=False, x_log_scale=False)


### CartPole (contínuo)

In [ ]:
ENV = gym.make("CartPole-v1")
NUM_EPISODES = 15000

results = []

#'bin1': 93,
#'bin2': 43,
#'bin3': 6,
#'bin4': 76,
#'epsilon': 0.015607854707332426,
#'gamma': 0.9134000716662984,
#'learning_rate': 0.11056680467861989}
wrapped_env = DiscreteObservationWrapper(ENV, [93,43,6,76])
results.append( repeated_exec(3, f"Q-Learning (Lucas)", run_qlearning, wrapped_env, NUM_EPISODES, 0.110, 0.913, 0.015) )
clear_output()

# Melhores ????
#wrapped_env = DiscreteObservationWrapper(ENV, [54,70])
#results.append( repeated_exec(3, f"Exp-Sarsa (Savio)", run_expected_sarsa, wrapped_env, NUM_EPISODES, 0.679, 0.95, 0.137) )
clear_output()

# Melhores parâmetros: {'bins1': 23, 'bins2': 37, 'bins3': 30, 'bins4': 38, 'epsilon': 0.05396213628972839, 'learning_rate': 0.5536537056140982}
wrapped_env = DiscreteObservationWrapper(ENV, [93,43,6,76])
results.append( repeated_exec(3, f"MonteCarlo2 (Giulia)", run_montecarlo2, wrapped_env, NUM_EPISODES, 0.553, 0.95, 0.053) )
clear_output()

plot_multiple_results(results, cumulative=False, x_log_scale=False)